In [1]:
import numpy as np
import pandas as pd
import sqlalchemy

from database_credentials import get_database_url
from datetime import datetime, timedelta

In [9]:
def drop_features(data):
    """Drop features not related to turnip prices with
    the exception of In-Game Name and Island
    
    Args: 
        data (dataframe): 
            original data from Maddox Knight's Turnip Mafia
    
    Returns:
        data (dataframe): 
            contains In-Game Name, Island along with the 
            buying and selling prices of turnips
    """
    
    column_names = [
        'In-Game Name', 'Island', 'Buy Price', 'Mon AM', 
        'Mon PM', 'Tue AM', 'Tue PM', 'Wed AM', 'Wed PM', 
        'Thu AM', 'Thu PM', 'Fri AM','Fri PM','Sat AM','Sat PM'
    ]
    data = data[column_names]
    return data

def mask_invalid_names(data):
    """Mask rows in which In-Game Name or Island are invalid
    
    Args: 
        data (dataframe): 
            contains data associated with invalid In-Game Names 
            or Island
    
    Returns:
        data (dataframe): 
            contains data associated with both a valid In-Game Name 
            and Island
    """
    
    invalid_ign_mask = data['In-Game Name'].notna()
    invalid_island_mask = data['Island'].notna()
    valid_data = data[invalid_ign_mask & invalid_island_mask].copy()
    valid_data.reset_index(drop=True, inplace=True)
    return valid_data

def convert_entry_to_float(entry):
    """Convert a price entry to float while replacing invalid
    prices with NaN
    
    Args:
        entry (str/float):
            entry to be converted
    
    Returns:
        convert_entry (float): 
            entry as a float or np.nan 
    """
    
    try: 
        converted_entry = float(entry)
    except:
        converted_entry = np.nan
    return converted_entry

def preprocess(data):
    """Preprocess data by dropping features, masking invalid 
    In-game Names, Islands along with formatting turnip prices
    
    Args: 
        data (dataframe): 
            original data from Maddox Knight's Turnip Mafia
        
    Returns:
        preprocessed_data (dataframe): 
            contains In-Game Name, Island along with the 
            buying and selling prices of turnips
    """
    
    preprocessed_data = drop_features(data)
    preprocessed_data = mask_invalid_names(preprocessed_data)
    
    price_columns = preprocessed_data.columns[2:]
    preprocessed_data[price_columns] = (
        preprocessed_data[price_columns].applymap(convert_entry_to_float)
    )
        
    return preprocessed_data

def get_buy_date():
    """Get the turnip buy date

    Args:
        None
    
    Returns:
        buy_dt (datetime): turnip buy date in %Y-%m-%d format
    """
    
    date_is_invalid = True
    while date_is_invalid:
        try:
            buy_date = input("Buy Date of Turnips (yyyy-mm-dd): ")
            buy_dt = datetime.strptime(buy_date, '%Y-%m-%d')
            date_is_invalid = False
        except:
            print("Invalid date format.")

    return buy_dt

def get_price_data(data, price_column, date):
    """Get the prices from a column and format it into a 
    dataframe columns containing In-Game Name, Island, 
    price and date 
    
    Args: 
        data (dataframe): preprocessed data
        price_column (str): column containing prices
        date (datetime): date prices were observed
    
    Returns:
        prices (dataframe): 
            contains In-Game Name, Island, turnip prices and date 
    """
    
    column_names = data.columns[0:2].to_list()
    column_names.append(price_column)
    price_data = pd.DataFrame(data[column_names])
    price_data.dropna(inplace=True)
    price_data['timestamp_observed'] = date
    
    new_column_names = {
        'In-Game Name': 'user_id', 
        'Island': 'island_id',
        price_column: 'price'
    }
    
    price_data.rename(columns=new_column_names, inplace=True)
    return price_data

def get_buy_sell_price_data(data, buy_date):
    """Get the buying and selling prices of turnips from data
    
    Args:
        data (dataframe): preprocessed data
        buy_date (datetime): buy date of turnips in yyyy-mm-dd format
    
    Returns:
        buy_prices (dataframe): buying prices of turnips
        sell_prices (dataframe): selling prices of turnips
    """
    
    buy_data = get_price_data(data, 'Buy Price', buy_date)
    
    # slice Mon AM, Mon PM, ... Sat AM, Sat PM
    sell_columns = data.columns[3:]
    first_sell_dt = buy_date + timedelta(days=1)
    
    # list comp yyyy-mm-dd 00:00:00 for AM and 
    # yyyy-mm-dd 12:00:00 for PM times
    sell_times = [
        first_sell_dt + timedelta(hours=12*elapsed_interval) 
        for elapsed_interval in range(sell_columns.shape[0])
    ]
    
    # get data for each column and datetime pair
    sell_data = [
        get_price_data(data, column, date) 
        for column, date in zip(sell_columns, sell_times)
    ]
    sell_data = pd.concat(sell_data)
    
    return buy_data, sell_data

In [15]:
# data = pd.read_csv('data/week1.csv', skiprows=[0]) # 2020-04-12
data = pd.read_csv('data/week2.csv', skiprows=[0, 1]) # 2020-04-19

In [16]:
preprocessed_data = preprocess(data)

In [17]:
display(preprocessed_data.head(3))

,In-Game Name,Island,Buy Price,Mon AM,Mon PM,Tue AM,Tue PM,Wed AM,Wed PM,Thu AM,Thu PM,Fri AM,Fri PM,Sat AM,Sat PM
0,Maddox,Knight,100.0,87.0,83.0,79.0,75.0,70.0,67.0,NaN,NaN,299.0,148.0,106.0,68.0
1,Mercure,Ronron,103.0,92.0,89.0,86.0,82.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Karen,CinderMar,100.0,89.0,83.0,NaN,98.0,198.0,515.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
buy_date = get_buy_date()
buy_data, sell_data = get_buy_sell_price_data(preprocessed_data, buy_date)

Buy Date of Turnips (yyyy-mm-dd): 2020-04-19


In [19]:
display(buy_data.head(3))

,user_id,island_id,price,timestamp_observed
0,Maddox,Knight,100.0,2020-04-19
1,Mercure,Ronron,103.0,2020-04-19
2,Karen,CinderMar,100.0,2020-04-19


In [12]:
database_url = get_database_url()
engine = sqlalchemy.create_engine(database_url)
connection = engine.connect()

In [20]:
buy_data.to_sql('buy_data', connection, if_exists='append', index=False)
sell_data.to_sql('sell_data', connection, if_exists='append', index=False)

In [ ]:
connection.close()
engine.dispose()